# Workshop n° 4: Parametric House Roofs

# Roberto Nunziato, matricola 438778

# Descrizione

L'obiettivo di questo workshop è quello di realizzare un modello di tetto. Nella realizzazione, sono state prese come riferimento le seguenti immagini:

1) Modello di tetto semplice

![img text](tetto1.jpg)

2) Modello di tetto complesso

![img text](tetto2.jpg)

# Librerie importate

In [1]:
from pyplasm import *
from fpformat import *

Evaluating fenvs.py..
...fenvs.py imported in 0.008488 seconds


Per prima cosa, costruiamo in pianta i modelli dei tetti di riferimento indicando i loro vertici e le celle: 

- <b>tettoSemplice</b> sarà il modello della figura n°1                                                                         
- <b>tettoComplesso</b> sarà il modello della figura n°2

In [2]:
vertici = [[1,1,0],[9,1,0],[9,5,0],[1,5,0],[3,3,0],[3,3,2],[7,3,0],[7,3,2]]
celle = [[1,2,5,6,7,8],[1,4,5,6],[2,3,7,8],[3,4,5,6,7,8]]
tettoSemplice = MKPOL([vertici,celle,None])

vertici2 = [[0,1.5,0],[0,3,0],[0,3,1],[0,4.5,0],[3,1.5,0],[3.75,3,0],[3.75,3,1],[3,4.5,0],
            [3,0,0],[4,0,0],[4,0,1.5],[5,0,0],[5,7,0],[4,7,0],[4,7,1.5],[3,7,0]]
celle2 = [[1,2,3,4],[5,6,7,8],[1,5,6,7,3,2],[2,3,4,6,7,8],
            [9,10,11,12],[13,14,15,16],[14,15,16,9,10,11],[10,11,12,13,14,15]]
tettoComplesso = MKPOL([vertici2,celle2,None])

La funzione principale è <b>ggpl_roof_builder</b>: dato un oggetto HPC di input (il disegno del tetto), attraverso le funzioni <b>UKPOL</b> e <b>SKEL_2</b> vengono presi tutti i punti dell'oggetto che sono divisi in due liste, quella dei vertici (<b>roofVertex</b>) e quella delle facce (<b>roofCells</b>).
Le liste verranno successive ripulite da eventuali doppioni ed i valori numerici arrotondati alla terza cifra decimale; le liste finali (<b>vertex</b> e <b>cells</b>) saranno quindi usate per definire la copertura  che verrà appoggiata sullo scheletro del tetto (<b>skeleton1</b>). <b>struttura</b> e <b>skeleton1</b> sono i modelli che verranno passati al metodo <b>STRUCT</b> per andare a definire l'oggetto HPC finale (<b>roof</b>), il quale verrà visualizzato attraverso la funzione <b>VIEW</b>.

In [3]:
def ggpl_roof_builder(hpc):
    skeleton1 = OFFSET([0.1,0.1,0.1])(SKEL_1(hpc)) #1-scheletro della struttura del tetto
    skeleton2 = UKPOL(SKEL_2(hpc)) #2-scheletro della struttura del tetto
    roofVertex = skeleton2[0] #lista dei vertici
    roofCells = skeleton2[1]  #lista delle celle
    
    #Ciclo la lista delle celle, per andare a sostituire agli indici dei punti,
    #le rispettive coordinate [x,y,z]
    cells2vertex = []
    #cell è una tripla [indiceX,indiceY,indiceZ]
    for cell in roofCells:
        l = []
        #index è iterativamente indiceX,indiceY ed indiceZ,a cui sostituisco le coordinate effettive
        for index in cell:
            l.append(roofVertex[index-1])
        cells2vertex.append(l)
    
    #Cicliamo la lista costruita al passo precedente, per andare ad arrotondare tutti i valori
    #alla terza cifra decimale
    vertexCleanFinal = []
    for v in cells2vertex:
        lista = []
        for i in v:
            l = []
            for n in i:
                n = float(fix(n,3))
                l.append(n)
            lista.append(l)    
        vertexCleanFinal.append(lista)  

    #In questo ciclo eliminiamo dalla lista precedente i doppioni
    vertexClean = [] #lista pulita di vertex(senza doppioni)
    for elem in vertexCleanFinal:
        for i in elem:
            if i not in vertexClean:
                vertexClean.append(i)
            else:
                i = vertexClean.index(i)
    
    #In questo ciclo, data la lista dei vertici ripulita da ogni doppione,
    #ricostruiamo la lista delle celle facendo il procedimento opposto a quello fatto
    #nel primo passo
    cells = []  #lista finale di celle
    for v in vertexCleanFinal:
        l = []
        for i in v:
            l.append(vertexClean.index(i)+1)
        cells.append(l)
    
    #Costruiamo la lista finale di vertici che verrà usata per la costruzione della struttura finale,
    #andando a prendere i soli punti perimetrali
    vertex = [] #lista finale di vertici
    trovato = 0
    for v in vertexClean:
        if v[2]>0: #z!=0
            vertex.append(v)
        else: #z==0
            for v2 in vertexClean:
                if v2[2]>0: #andiamo a vedere se c'è un punto sopra a quello con coordinata z=0, in quel caso
                            #lo prendiamo
                    if v2[0]==v[0] and v2[1]==v[1]:
                        vertex.append(v2)
                        trovato = 1
            if not trovato: #se il vertice non ha un punto superiore, allora è un punto perimetrale, quindi lo
                            #prendiamo
                vertex.append(v)
            trovato = 0

    struttura=MKPOL([vertex,cells,None])
    struttura=STRUCT([T(3)(0.1),struttura])
    struttura=OFFSET([0.1,0.1,0.1])(struttura)
    roof= STRUCT([struttura,skeleton1])            
            
    return VIEW(roof)       

In [4]:
ggpl_roof_builder(tettoComplesso)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10dfedb70> >

# Risultato ottenuto

1) Oggetto HPC del <b>tettoSemplice</b>

![alt-text](tettoSemplice1.png)
![alt-text](tettoSemplice2.png)

2) Oggetto HPC del <b>tettoComplesso</b>

![alt-text](tettoComplesso1.png)
![alt-text](tettoComplesso2.png)